In [2]:
# Scientific imports

%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astroquery.mast import Observations
from astroquery.mast import Catalogs

In [3]:
# General imports

import csv, math, os, os.path, sys
import pandas as pd
import seaborn as sb
from sklearn import metrics
from IPython.display import display

sb.set()

In [17]:
# Reading which LC datafiles we have into a list

fitsList = []
fits_directories = [x[0] for x in os.walk('.')]

for fits_root, fits_dirs, fits_files in os.walk('.'):
    for fits_file in fits_files:
        fullpath = os.path.join(fits_root, fits_file)
        if (os.path.splitext(fullpath.lower())[1]).endswith('.fits'):
            fitsList.append(fullpath)
print("Number of FITS files: {}".format(len(fitsList)))
for num, i in enumerate(fitsList):
    print ("{}: {}".format(num, i))

Number of FITS files: 27
0: ./kplr010666592-2009131105131_llc.fits
1: ./mastDownload/IUE/lwp13058/lwp13058mxlo_vo.fits
2: ./mastDownload/TESS/tess2018206045859-s0001-0000000270501383-0120-s/tess2018206045859-s0001-0000000270501383-0120-s_lc.fits
3: ./mastDownload/TESS/tess2018206045859-s0001-0000000270501383-0120-s/tess2018206045859-s0001-0000000270501383-0120-s_tp.fits
4: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206045859-s0001-0000000097409519-0120-s_lc.fits
5: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206045859-s0001-0000000097409519-0120-s_tp.fits
6: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206190142-s0001-s0001-0000000097409519-00106_dvt.fits
7: ./mastDownload/TESS/tess2018206045859-s0001-0000000099837626-0120-s/tess2018206045859-s0001-0000000099837626-0120-s_tp.fits
8: ./mastDownload/TESS/tess2018206045859-s0001-0000000099837626-0120-s/tess2018206045859-s0001-0000000099837626-0

Now that we have a list of all FITS files we have, I can select any one of them (in this list) and use that as the filename. Now, let's have a look at a FITS file, to check if it's a Light Curve (LC) or an image.

In [96]:
file = fitsList[0]
check = fits.info(file)

#filesize = (1/1024)*os.stat(file).st_size
#print ("{} kb".format(filesize))

Filename: ./kplr010666592-2009131105131_llc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   ()      
  1  LIGHTCURVE    1 BinTableHDU    155   476R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        48   (12, 10)   int32   


In [97]:
# The following line of code gives us the header values
fitsHeaders = fits.getheader(file)
fitsHeaders

SIMPLE  =                    T / conforms to FITS standards                     
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T / file contains extensions                       
NEXTEND =                    2 / number of standard extensions                  
EXTNAME = 'PRIMARY '           / name of extension                              
EXTVER  =                    1 / extension version number (not format version)  
ORIGIN  = 'NASA/Ames'          / institution responsible for creating this file 
DATE    = '2015-08-26'         / file creation date.                            
CREATOR = '793561 FluxExporter2PipelineModule' / pipeline job and program used t
PROCVER = 'svn+ssh://murzim/repo/soc/tags/release/9.3.21 r60196' / SW version   
FILEVER = '6.1     '           / file format version                            
TIMVERSN= 'OGIP/93-003'     

In [114]:
def areAllHeadersPresent(heads):
    """
    A function to make sure that all headers required for plotting a Light Curve
    are present, and alert the user if any are missing (and which ones)
    """
    
    allgood = True
    neededHeaders = ['TEFF', 'LOGG', 'TESSMAG',
                     'TPERIOD', 'TDUR', 'TEPOCH', 'TDEPTH',
                     'TIME', 'PHASE', 'LC_INIT', 'MODEL_INIT']

    for i in range (len(neededHeaders)):
        try:
            heads[neededHeaders[i]]
        except:
            #print("Header {} not present!".format(neededHeaders[i]))
            allgood = False
        #else:
        #    print("{}: {}".format(neededHeaders[i], heads[neededHeaders[i]]))
    return allgood

#areAllHeadersPresent(fitsHeaders)

In [119]:
if areAllHeadersPresent(fitsHeaders)==False:
    print("Not all required Headers are present in this file!")
else:

    with fits.open(file, mode="readonly") as hdulist:


        # Extract stellar parameters from the primary header.  We'll get the effective temperature, surface gravity,
        # and TESS magnitude.
        star_teff = hdulist[0].header['TEFF']
        star_logg = hdulist[0].header['LOGG']
        star_tmag = hdulist[0].header['TESSMAG']

        # Extract some of the fit parameters for the first TCE.  These are stored in the FITS header of the first
        # extension.
        period = hdulist[1].header['TPERIOD']
        duration = hdulist[1].header['TDUR']
        epoch = hdulist[1].header['TEPOCH']
        depth = hdulist[1].header['TDEPTH']

        # Extract some of the columns of interest for the first TCE signal.  These are stored in the binary FITS table
        # in the first extension.  We'll extract the timestamps in TBJD, phase, initial fluxes, and corresponding
        # model fluxes.
        times = hdulist[1].data['TIME']
        phases = hdulist[1].data['PHASE']
        fluxes_init = hdulist[1].data['LC_INIT']
        model_fluxes_init = hdulist[1].data['MODEL_INIT']

    # First sort the phase and flux arrays by phase so we can draw the connecting lines between points.
    sort_indexes = np.argsort(phases)

    # Start figure and axis.
    fig, ax = plt.subplots(figsize=(12,4))

    # Plot the detrended fluxes as black circles.  We will plot them in sorted order.
    ax.plot(phases[sort_indexes], fluxes_init[sort_indexes], 'ko',
           markersize=2)

    # Plot the model fluxes as a red line.  We will plot them in sorted order so the line connects between points cleanly.
    ax.plot(phases[sort_indexes], model_fluxes_init[sort_indexes], '-r')

    # Let's label the axes and define a title for the figure.
    fig.suptitle('TIC 100100827 - Folded Lightcurve And Transit Model.')
    ax.set_ylabel("Flux (relative)")
    ax.set_xlabel("Orbital Phase")

    # Let's add some text in the top-right containing some of the fit parameters.
    plt.text(0.2, 0.012, "Period = {0:10.6f} days".format(period))
    plt.text(0.2, 0.010, "Duration = {0:10.6f} hours".format(duration))
    plt.text(0.2, 0.008, "Depth = {0:10.6f} ppm".format(depth))
    plt.text(0.45, 0.012, "Star Teff = {0:10.6f} K".format(star_teff))
    plt.text(0.45, 0.010, "Star log(g) = {0:10.6f}".format(star_logg))

    plt.show()

Not all required Headers are present in this file!
